必要なライブラリのインポート

In [96]:
import tensorflow as tf
from tensorflow.keras import layers, models,optimizers,losses
import scipy.io
import numpy as np

GoogleDriveをマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


データの読み込み

In [97]:
data=scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/ml_class/usps_resampled/usps.mat')

dict_keys(['__header__', '__version__', '__globals__', 'test', 'trai', 'test_label', 'trai_label'])

データの整形

In [98]:
trai=data['trai'].reshape(1,-1,order='F').reshape(4649,16,16)
test=data['test'].reshape(1,-1,order='F').reshape(4649,16,16)
trai_label = data['trai_label']
test_label = data['test_label']

(4649, 16, 16)

モデルの構築

In [104]:
model=models.Sequential()
model.add(layers.Reshape((16,16,1),input_shape=(16,16)))

# 畳み込み処理１回目（Conv）
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(16, 16, 1)))

# 畳み込み処理２回目（Conv→Pool）
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# 全結合層と出力層（Flatten→Dense→Dropout→Dense）
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))

#モデルの概要を表示
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_30 (Reshape)         (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 12, 12, 32)        832       
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 128)               102528    
_________________________________________________________________
dropout_44 (Dropout)         (None, 128)             

モデルのコンパイルと学習

In [106]:
model.compile(optimizer=optimizers.RMSprop(),
                loss=losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
model.fit(trai,trai_label,batch_size=32,epochs=20,validation_data=(test,test_label))

Epoch 1/20
146/146 [==============================] - 4s 21ms/step - loss: 0.6079 - accuracy: 0.8133 - val_loss: 0.2300 - val_accuracy: 0.9307
Epoch 2/20
146/146 [==============================] - 3s 20ms/step - loss: 0.2160 - accuracy: 0.9333 - val_loss: 0.1720 - val_accuracy: 0.9492
Epoch 3/20
146/146 [==============================] - 3s 20ms/step - loss: 0.1529 - accuracy: 0.9542 - val_loss: 0.1115 - val_accuracy: 0.9652
Epoch 4/20
146/146 [==============================] - 3s 20ms/step - loss: 0.1182 - accuracy: 0.9660 - val_loss: 0.1200 - val_accuracy: 0.9615
Epoch 5/20
146/146 [==============================] - 3s 20ms/step - loss: 0.0936 - accuracy: 0.9699 - val_loss: 0.0936 - val_accuracy: 0.9718
Epoch 6/20
146/146 [==============================] - 3s 20ms/step - loss: 0.0847 - accuracy: 0.9753 - val_loss: 0.0996 - val_accuracy: 0.9695
Epoch 7/20
146/146 [==============================] - 3s 20ms/step - loss: 0.0741 - accuracy: 0.9772 - val_loss: 0.0735 - val_accuracy: 0.9766

モデルの評価

In [107]:
loss,accuracy = model.evaluate(test, test_label)

146/146 [==============================] - 1s 5ms/step - loss: 0.0740 - accuracy: 0.9824


混同行列の作成

In [108]:
test_pred = tf.argmax(model.predict(test), axis=-1).numpy()
CONF=tf.math.confusion_matrix(test_label, test_pred)
print(CONF)
print('accuracy=',round(accuracy*100,2),sep='')

tf.Tensor(
[[772   0   4   0   4   3   1   0   1   1]
 [  0 646   0   0   0   0   0   0   1   0]
 [  3   1 442   0   4   2   0   2   0   0]
 [  0   0   1 405   0  11   0   0   0   1]
 [  0   1   0   0 441   1   0   0   0   0]
 [  0   0   0   3   0 349   1   1   0   1]
 [  1   0   0   0   5   2 406   0   0   0]
 [  0   1   2   0   1   0   0 394   1   3]
 [  3   2   1   0   2   1   1   1 320   0]
 [  0   0   0   0   1   2   0   3   1 392]], shape=(10, 10), dtype=int32)
accuracy=98.24
